In [1]:
import pathlib

import pandas as pd

In [2]:
file_path = pathlib.Path(
    "../../data/5.converted_data/sum_aggregated_data.parquet"
).resolve(strict=True)
df = pd.read_parquet(file_path)
df.head()

,Metadata_genotype,Metadata_replicate,Metadata_side,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,high,1,L,-0.178797,-0.178797,0.003558,-0.386796,0.280118,0.037998,0.434046,...,-0.627762,-0.644990,-0.903354,-0.890939,-0.896995,-0.902870,-0.896145,-0.898448,-0.898880,-0.896609
1,high,1,R,1.371763,1.371763,0.538195,0.288322,0.501618,4.163833,-1.361920,...,0.419551,0.406918,-0.188358,-0.159114,-0.189615,-0.186308,-0.218981,-0.225754,-0.219073,-0.225134
2,high,10,L,1.706234,1.706234,2.652373,3.280425,-1.992966,0.184630,-1.404376,...,0.841475,0.834574,1.149186,1.152200,1.078905,1.144848,1.040009,1.025740,1.045492,1.027617
3,high,10,R,0.771674,0.771674,-1.332747,0.164304,0.244371,0.173760,-1.885144,...,0.452480,0.436138,0.112178,0.131982,0.108602,0.125700,0.075717,0.071207,0.076477,0.067922
4,high,11,L,2.180858,2.180858,2.142686,1.838038,-0.077515,0.008650,0.876597,...,0.494813,0.490844,-0.016140,0.001041,-0.021997,-0.020992,-0.033934,-0.042635,-0.034983,-0.038763


We want to remove CellProfiler features that do not accurately contribute to the shape variance.


Spatial moment features are removed as they are not invariant to rotation and translation.

Central moment features are removed as measurements of size is not of interest.

Normalized moment features are removed as they do not contribute to the shape variance.

EulerNumber is removed as we are not concerned with the number of holes in the shape.

Extent is removed as we do not care about the ratio of pixels in the bounding box to the pixels in the shape.

Orientation is removed as we are not concerned with the angle of the major axis of the shape.

Zernike features are removed as they are not invariant to rotation and translation.

Texture features are removed as we are not interested in the texture of the shape.

In [3]:
list_to_drop = [
    "SpatialMoment",
    "CentralMoment",
    "NormalizedMoment",
    "EulerNumber",
    "Extent",
    "Orientation",
    "Zernike",
    "Texture",
]

In [4]:
# drop columns that contain Metadata
df1 = df.copy()
df1 = df1.drop(columns=[col for col in df1.columns if "Metadata" in col])
columns = df1.columns.to_list()
print(df1.shape)
# if column contains any of the strings in list_to_drop, drop it
for col in list_to_drop:
    columns = [c for c in columns if col not in c]
df1 = df1[columns]
print(df1.shape)
df1["Metadata_genotype"] = df["Metadata_genotype"]
df1["Metadata_replicate"] = df["Metadata_replicate"]

(83, 244)
(83, 64)


In [5]:
df1.head()

,AreaShape_Area,AreaShape_Compactness,AreaShape_ConvexArea,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_FormFactor,AreaShape_HuMoment_0,AreaShape_HuMoment_1,AreaShape_HuMoment_2,AreaShape_HuMoment_3,...,RadialDistribution_MeanFrac_OP_1of4,RadialDistribution_MeanFrac_OP_2of4,RadialDistribution_MeanFrac_OP_3of4,RadialDistribution_MeanFrac_OP_4of4,RadialDistribution_RadialCV_OP_1of4,RadialDistribution_RadialCV_OP_2of4,RadialDistribution_RadialCV_OP_3of4,RadialDistribution_RadialCV_OP_4of4,Metadata_genotype,Metadata_replicate
0,-0.178797,2.083799,0.496298,-3.863745,-0.015242,-1.467570,-0.617409,-0.702934,-0.638018,-0.634731,...,1.037015,1.022628,0.637994,-0.716125,-1.377808,-1.241646,-1.116868,-1.674243,high,1
1,1.371763,-1.160529,1.248658,-0.307351,1.363698,1.460106,-0.821478,-0.636947,-0.966086,-0.650675,...,-0.828175,-1.100594,-0.843636,1.077265,-0.282041,-0.254773,0.382132,0.779949,high,1
2,1.706234,-1.094663,1.526738,-0.364618,1.604017,1.301326,-0.850517,-0.643917,-0.930476,-0.658509,...,-1.316829,-1.414181,-1.081362,1.472547,0.721204,1.019337,1.413652,1.432583,high,10
3,0.771674,-1.074951,0.753993,0.169009,0.892807,1.255925,-0.679950,-0.573659,-0.915379,-0.624424,...,-1.180881,-1.336964,-1.518430,1.639641,1.061081,0.230001,0.798123,1.372333,high,10
4,2.180858,-0.296613,2.408620,-2.937313,1.923791,-0.006871,-0.933497,-0.701417,-0.947919,-0.655172,...,1.434761,0.333079,-0.424506,-0.325401,1.638487,1.213389,0.152224,0.230334,high,11


In [6]:
# save the dataframe
df_path = pathlib.Path(
    "../../data/5.converted_data/sum_aggregated_manual_feature_selected_data.parquet"
)
df1.to_parquet(df_path)